In [1]:
import pandas as pd
import re
import json
import itertools
import pickle

In [2]:
def join_block(text):
    # Transformar texto em linhas de folha de documento em texto corrido em parágrafo

    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]

    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)

    return " ".join(m_words)

def clean_text(text, filter_patterns, rep=""):
    filter_patterns = "|".join(filter_patterns)

    text = pd.Series(text).str.replace(filter_patterns, rep, regex=True)[0]

    return join_block(text)

In [ ]:
def get_filtered_blocks(df, patterns_clean):
    # Separar texto em blocos e limpá-los
    
    pos_texts = []

    # Identificação de blocos é feita com as tags xxbob e xxeob
    for number, text in df[["number", "text"]].itertuples(index=False):
        begin = list(re.finditer("xxbob", text))
        end = list(re.finditer("xxeob", text))
        pos_texts.append([number, text, begin, end])
        
    full_blocks = []

    for p in pos_texts:
        blocks = []

        number = p[0]
        text = p[1]
        begin = p[2]
        end = p[3]

        if (number % 50) == 0:
            print(number)

        for i in range(len(begin)):
            block = text[begin[i].start():end[i].end()]

            for i in range(len(patterns_clean["patterns"])):
                block = clean_text(text=block, filter_patterns=patterns_clean["patterns"][i], rep=patterns_clean["rep"][i])

            blocks.append(block)

        full_blocks.append([number, blocks])
    
    return full_blocks

In [ ]:
def find_matching(full_blocks, editais):
    # Busca de matchings entre SIS Editais e os blocos encontrados e filtrados
    
    encontrados = []

    for f in full_blocks:
        number = f[0]
        blocks = f[1]

        blocks_mod = pd.Series(blocks).str.lower()
        
        ementas_obj = editais[editais["n_dodf"] == number]["Ementa Objeto"].str.lower()
        processos_obj = editais[editais["n_dodf"] == number]["ProcessoGDF"].astype(str)
        editais_obj = editais[editais["n_dodf"] == number]["Edital"].astype(str)
        
        for ementa, processo, edital in zip(ementas_obj, processos_obj, editais_obj):
            main_rule = re.escape(ementa)
            sec_rule = re.escape(processo)
            third_rule = re.escape(edital)
            
            # A prioridade é dada a Ementa Objeto, depois ProcessoGDF e, por fim, Edital
            if blocks_mod.str.contains(main_rule).sum() == 1:
                encontrados.append([number, "ementa", ementa, blocks_mod[blocks_mod.str.contains(main_rule)].iloc[0]])
            elif blocks_mod.str.contains(sec_rule).sum() == 1:
                encontrados.append([number, "processo", processo, blocks_mod[blocks_mod.str.contains(sec_rule)].iloc[0]])
            elif blocks_mod.str.contains(third_rule).sum() == 1:
                encontrados.append([number, "edital", edital, blocks_mod[blocks_mod.str.contains(third_rule)].iloc[0]])
                
    return encontrados

In [3]:
def merge_data(data):
    # Concatenar os dados do DODF, juntando todas as páginas em uma linha
    
    return data.sort_values(by=["file_name", "page"]).groupby(["file_name", "number", "day", "month", "year"], as_index=False).agg({'text': '\n'.join})

In [ ]:
def get_index(row, editais):
    # Pegar index correspondente ao ato nos SIS Editais

    if row["selection_type"] == "ementa":
        return editais[(row["year"] == editais["year"]) & (row["n_dodf"] == editais["n_dodf"]) & (row["selection_rule"] == editais["Ementa Objeto"].str.lower())].index.values
    elif row["selection_type"] == "processo":
        return editais[(row["year"] == editais["year"]) & (row["n_dodf"] == editais["n_dodf"]) & (row["selection_rule"] == editais["ProcessoGDF"])].index.values
    elif row["selection_type"] == "edital":
        return editais[(row["year"] == editais["year"]) & (row["n_dodf"] == editais["n_dodf"]) & (row["selection_rule"] == editais["Edital"])].index.values

    return None

In [4]:
# Regras de limpeza, separadas por tipo de substituição a fazer
patterns_clean = json.loads(r'{"patterns": [["\\nP\u00c1GINA\\s([0-9]{1,5})", "\\nDI\u00c1RIO\\sOFICIAL\\sDO\\sDISTRITO\\sFEDERAL", "\\nN\u00ba(.+?)2([0-9]{3})", "\\nxx([a-z]{0,10}) Di\u00e1rio Oficial do Distrito Federal xx([a-z]{0,10})", "\\nDi\u00e1rio Oficial do Distrito Federal", "Documento assinado digitalmente conforme MP n\u00ba 2.200-2 de 24/08/2001, que institui a", "Infraestrutura de Chaves P\u00fablicas Brasileira ICP-Brasil", "Este documento pode ser verificado no endere\u00e7o eletr\u00f4nico", "http://wwwin.gov.br/autenticidade.html", "pelo c\u00f3digo ([0-9]{15,18})", "\\nDocumento assinado digitalmente, original em https://www.dodf.df.gov.br", "http:/Awwwin.gov.br/autenticidade.html", "Documento assinado digitalmente conforme MP n\u00ba 2.200-2 de 24/08/2001,", "\\nque institui a\\n", "\\nhttp://www.in.gov.br/autenticidade.html", "\\nhttp://www.in.gov.brautenticidade html", "Documento assinado digitalmente conforme MP n 2.200-2 de 24/08/2001, que institui a .", "http://www.in.gov.brautenticidade html,", "xx([a-z]{1,10}) ", " xx([a-z]{1,10})", "xx([a-z]{1,10})"], ["\\n-\\n", "\\n- -\\n", "\\n- - -\\n", "\\n[\\.\\,\\-\\\u2014]\\n", "\u2014 -", ". -", "\\r[\\.\\,\\-\\\u2014]\\r", "\\n-\\r", "\\r"]], "rep": [" ", "\n"]}')

In [5]:
sis_editais = pd.read_csv("siseditais_alterado.csv", encoding = 'latin1', sep=";")
sis_editais["Modalidade"] = sis_editais["Modalidade"].str.replace("PREG?O ELETR?NICO", "PREGÃO ELETRÔNICO", regex=False)

In [6]:
sis_editais_mod = sis_editais.copy()

In [7]:
# Pegar data dos DODFs de cada linha
sis_editais_mod["day"] = sis_editais_mod["Publicação"].str.split("/").str[0]
sis_editais_mod["month"] = sis_editais_mod["Publicação"].str.split("/").str[1]
sis_editais_mod["year"] = sis_editais_mod["Publicação"].str.split("/").str[2]

In [8]:
# Remover linhas mal-formatadas
sis_editais_mod = sis_editais_mod[(sis_editais_mod["Dodf"].str.strip() != "") & (~sis_editais_mod["Dodf"].isnull())].reset_index(drop=True)

In [9]:
# Pegar número do DODF por linha
sis_editais_mod["n_dodf"] = sis_editais_mod["Dodf"].str.split("/").str[0].str.strip().astype(int)

In [10]:
# Só há dados consistentes e confiáveis entre 2011 e 2021
sis_editais_mod = sis_editais_mod[~sis_editais_mod["year"].isnull()]
sis_editais_mod = sis_editais_mod[(sis_editais_mod["year"].astype(int) > 2010) & (sis_editais_mod["year"].astype(int) < 2022)].reset_index(drop=True)
sis_editais_mod["year"] = sis_editais_mod["year"].astype(int)

In [16]:
full_matchings = []

for year_sis in sis_editais_mod["year"].unique():
    print(year_sis)
    
    # data_dodf contém a base de DODFs do Fabrício, segmentada por ano
    data_dodf = pd.read_parquet("/home/ciri/Desktop/sofia/unb/knedle/anotacoes_manuais_tcdf/dodfs_parquet/df_{}.parquet.gzip".format(str(year_sis)))
    data_dodf = merge_data(data_dodf)
    
    sis_editais_temp = sis_editais_mod[sis_editais_mod["year"] == year_sis]
    
    full_blocks_data = get_filtered_blocks(data_dodf, patterns_clean)
    matchings_temp = find_matching(full_blocks_data, sis_editais_temp)
    
    full_matchings.append([year_sis, matchings_temp])
    
    print()

2021
50
100
150

2020
50
100
150
200

2019
50
100
150
200

2018
50
100
150
200

2017
50
100
150
200

2016
50
100
150
200

2015
50
100
150
200
250

2014
50
100
150
200
250

2013
50
100
150
200
250

2012
50
100
150
200
250



In [18]:
with open('full_matchings_v2_2.pickle', 'wb') as handle:
    pickle.dump(full_matchings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
with open('full_matchings_v2_2.pickle', 'rb') as handle:
    full_matchings = pickle.load(handle)

In [40]:
matchings_year_dfs = []

for matchings_year in full_matchings:
    n_dodf = [i[0] for i in matchings_year[1]]
    year = matchings_year[0]
    selection_type = [i[1] for i in matchings_year[1]]
    selection_rule = [i[2] for i in matchings_year[1]]
    act = [i[3] for i in matchings_year[1]]
    
    matchings_year_df = pd.DataFrame({
        "n_dodf": n_dodf,
        "year": year,
        "selection_rule": selection_rule,
        "selection_type": selection_type,
        "text": act
    })
    
    matchings_year_df["loc_index"] = matchings_year_df.apply(lambda row: get_index(row, sis_editais_mod), axis=1)
    matchings_year_dfs.append(matchings_year_df)
    
    print(year)

2021
2020
2019
2018
2017
2016
2015
2014
2013
2012


In [59]:
matchings_year_total = pd.concat(matchings_year_dfs, ignore_index=True)

In [72]:
# Remover casos de ambiguidade, com repetição de identificador
matchings_year_total = matchings_year_total[matchings_year_total["loc_index"].str.len() == 1].reset_index(drop=True)
matchings_year_total["loc_index"] = matchings_year_total["loc_index"].str[0]

In [91]:
sis_editais_mod["loc_index"] = sis_editais_mod.index.values

In [102]:
matchings_year_total_mod = matchings_year_total.merge(sis_editais_mod[["loc_index", "Tipo Objeto", "EspObjeto", "Licitante", "Interessado", "Responsável", "Modalidade"]], how="left", on=["loc_index"])

In [103]:
matchings_year_total_mod.to_csv("matchings_sis_dodf.csv", index=False)
sis_editais_mod.to_csv("sis_editais.csv", index=False)

In [105]:
len(matchings_year_total_mod)/len(sis_editais_mod)

0.7168220414464699